In [ ]:
import arcpy

# Ścieżki do danych
network_dataset = "ścieżka_do_network_dataset"
origins = "ścieżka_do_centroidów_heksów"
destinations = "ścieżka_do_centroidów_sąsiadów"
output_layer = "OD_Matrix_Layer"

# Tworzenie warstwy OD Cost Matrix
arcpy.na.MakeODCostMatrixLayer(network_dataset, output_layer, "TravelTime")

# Dodawanie punktów początkowych i końcowych
arcpy.na.AddLocations(output_layer, "Origins", origins)
arcpy.na.AddLocations(output_layer, "Destinations", destinations)

# Uruchomienie analizy
arcpy.na.Solve(output_layer)

# Eksport wyników
arcpy.management.CopyRows(f"{output_layer}\\Lines", "ścieżka_do_wyników")


In [ ]:
resolution = 8  # Możesz dostosować tę wartość


In [ ]:
import h3

def polygon_to_h3(geometry, resolution):
    # Konwertuj geometrię do formatu GeoJSON
    geojson = geometry.__geo_interface__
    # Generuj indeksy H3 pokrywające geometrię
    h3_indices = h3.polyfill(geojson, resolution, geo_json_conformant=True)
    return h3_indices


In [ ]:
from shapely.geometry import Polygon
import pandas as pd

# Lista do przechowywania wyników
h3_list = []

for idx, row in poligony.iterrows():
    geometry = row['geometry']
    h3_indices = polygon_to_h3(geometry, resolution)
    for h3_idx in h3_indices:
        h3_list.append({'h3_index': h3_idx, 'source_id': idx})


In [ ]:
def h3_index_to_polygon(h3_index):
    # Pobierz współrzędne wierzchołków heksagonu
    boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)
    # Utwórz poligon
    return Polygon(boundary)

# Dodaj geometrię do danych
for item in h3_list:
    item['geometry'] = h3_index_to_polygon(item['h3_index'])
